# 20 뉴스그룹 분류

In [41]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset= 'all',random_state= 156)

## 1. 데이터 탐색

In [4]:
news_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
news_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [8]:
pd.Series(news_data.target).value_counts().sort_index()

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

In [9]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

## 2. 훈련/ 테스트용 데이터 추출

In [10]:
train_news =  fetch_20newsgroups(
    subset= 'train', random_state=156,
    remove=('headers','footers','quotes') # 본인이 쓴 핵심 내용만 남기고 나머지 제거
)
X_train = train_news.data
y_train = train_news.target

In [12]:
print(train_news.data[0])



What I did NOT get with my drive (CD300i) is the System Install CD you
listed as #1.  Any ideas about how I can get one?  I bought my IIvx 8/120
from Direct Express in Chicago (no complaints at all -- good price & good
service).

BTW, I've heard that the System Install CD can be used to boot the mac;
however, my drive will NOT accept a CD caddy is the machine is off.  How can
you boot with it then?

--Dave



In [13]:
train_news.target[0], train_news.target_names[train_news.target[0]]

(4, 'comp.sys.mac.hardware')

In [14]:
test_news = fetch_20newsgroups(
    subset='test', random_state= 156,
    remove = ('headers','footers','quotes') 
)
X_test = test_news.data
y_test = test_news.target

In [16]:
len(X_train), len( y_test)

(11314, 7532)

## 3. 피쳐 벡터화와 머신러닝 모델학습 / 예측 / 평가

## case 1. CountVectorizer + LogisticRegRession


### X_train 과 X_test에 동일한 "Count Vectorizer" 를 사용해야 함! 
X_train  => Count Vectorizer => x_train_cv

X_test   => Count Vectorizer => x_test_cv

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test) # x_test는 x_train을 변환한 countvectorizer를 동일하게 사용하여야함.

In [19]:
X_train_cv.shape , X_test_cv.shape

((11314, 101631), (7532, 101631))

In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_cv, y_train)

C:\Users\CPB06GameN\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=300)

In [21]:
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [23]:
from sklearn.metrics import accuracy_score
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.5973181093998938

In [24]:
y_test[:5], pred[:5]


(array([ 4, 11,  1,  7,  8]), array([ 4, 11,  6,  7,  8]))

## Case 2. TfidVectorizer + LogisticRegression

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer()
tvect.fit(X_train)
X_train_tf = tvect.transform(X_train)
X_test_tf = tvect.transform(X_test)

In [26]:
X_train_tf.shape, X_test_tf.shape

((11314, 101631), (7532, 101631))

In [27]:
lr = LogisticRegression(max_iter = 300)
lr.fit(X_train_tf, y_train)

LogisticRegression(max_iter=300)

In [28]:
pred = lr.predict(X_test_tf)
accuracy_score(y_test, pred)

0.6736590546999469

In [29]:
y_test[:5], pred[:5]

(array([ 4, 11,  1,  7,  8]), array([5, 1, 1, 7, 8]))

## case3 . stop_words filtering, ngram_range = (1,2), max_df = 300

In [33]:
tvect2 = TfidfVectorizer(ngram_range=(1,2),max_df = 300, stop_words='english')
tvect2.fit(X_train)
X_train_tf2 = tvect2.transform(X_train)
X_test_tf2 = tvect2.transform(X_test)

In [36]:
X_train_tf2.shape, X_test_tf2.shape

((11314, 943453), (7532, 943453))

In [34]:
lr = LogisticRegression(max_iter=300)
lr.fit(X_train_tf2, y_train)
pred2 = lr.predict(X_test_tf2)
accuracy_score(y_test,pred2)

0.6922464152947424

## Case4. case3에서 LogisticRegression C값을 10으로

In [44]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(max_iter=300, C =10)
lr.fit(X_train_tf2, y_train)
pred2 = lr.predict(X_test_tf2)
accuracy_score(y_test,pred2)

0.7012745618693574

## 4.  Pipeline 과 GridSearchCV를 통한 하이퍼 파라메터 튜닝


In [62]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tvect', TfidfVectorizer(stop_words='english')),
    ('lr', LogisticRegression())
])

In [63]:
params = {
    'tvect__ngram_range': [(1,1), (1,2)],
    'tvect__max_df': [300, 700],
    'lr__C': [1, 10]
}

In [67]:
from sklearn.model_selection import GridSearchCV

grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3,
    scoring='accuracy', verbose=1, n_jobs=-1
)

In [70]:
%time grid_pipe.fit(X_train, y_train) # # %time 실행시간 같이 알려줌

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [1]:
print(grid_pipe.best_score_)
print(grid_pipe.best_params_)
pred = grid_pipe.best_estimator_.predict(X_test)
print(accuracy_score(y_test, pred))

NameError: name 'grid_pipe' is not defined